<a href="https://colab.research.google.com/github/MihaiDogariu/Keysight-Deep-Learning-Fundamentals--v2-/blob/main/scripts/Unit_6_Convnet_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=True) # Smaller batch for display

In [3]:
# Define the CNN model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(7 * 7 * 32, 10)  # Adjust input size based on image size

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 7 * 7 * 32)  # Adjust input size based on image size
        x = self.fc(x)
        return x

In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
# Initialize model, loss function, and optimizer
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training loop
for epoch in range(5):  # Reduced epochs for demonstration
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item()}')

Epoch 1, Batch 1, Loss: 2.3520889282226562
Epoch 1, Batch 101, Loss: 0.1847350001335144
Epoch 1, Batch 201, Loss: 0.14747481048107147
Epoch 1, Batch 301, Loss: 0.16619865596294403
Epoch 1, Batch 401, Loss: 0.03280072286725044
Epoch 1, Batch 501, Loss: 0.04150678217411041
Epoch 1, Batch 601, Loss: 0.05618174374103546
Epoch 1, Batch 701, Loss: 0.021272124722599983
Epoch 1, Batch 801, Loss: 0.16420528292655945
Epoch 1, Batch 901, Loss: 0.0819312110543251
Epoch 2, Batch 1, Loss: 0.14748288691043854
Epoch 2, Batch 101, Loss: 0.04580574855208397
Epoch 2, Batch 201, Loss: 0.051384419202804565
Epoch 2, Batch 301, Loss: 0.01880972646176815
Epoch 2, Batch 401, Loss: 0.054190658032894135
Epoch 2, Batch 501, Loss: 0.17219041287899017
Epoch 2, Batch 601, Loss: 0.02997857704758644
Epoch 2, Batch 701, Loss: 0.010102851316332817
Epoch 2, Batch 801, Loss: 0.1192546933889389
Epoch 2, Batch 901, Loss: 0.06266867369413376
Epoch 3, Batch 1, Loss: 0.031809061765670776
Epoch 3, Batch 101, Loss: 0.01577184721

In [ ]:
# Display some learned kernels
kernels = model.conv1.weight.data.cpu().numpy()

fig, axs = plt.subplots(4, 4, figsize=(8, 8))
for i in range(16):
    row = i // 4
    col = i % 4
    axs[row, col].imshow(kernels[i, 0], cmap='gray')
    axs[row, col].axis('off')
plt.show()

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

print("Accuracy is {}%".format(100 * correct / total))

In [ ]:
# Get a random batch from the test set
dataiter = iter(testloader)
images, labels = next(dataiter)

images = images.to(device)
labels = labels.to(device)

# Make predictions
outputs = model(images)
_, predicted = torch.max(outputs, 1)

# Display the images with true and predicted labels
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
for i in range(16):
    ax = axes[i // 4, i % 4]
    ax.imshow(images[i].squeeze().cpu().numpy(), cmap='gray')
    ax.set_title(f"True: {labels[i]}\nPredicted: {predicted[i]}")
    ax.axis('off')
plt.tight_layout()
plt.show()